In [24]:
push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main")
using DHC_2DUtils
using Profile
using BenchmarkTools
using FFTW
using HDF5
using Test
using SparseArrays
using Statistics
using Plots
using LinearAlgebra
using Measures
using Plots
using MLDatasets
using ImageCore
using Colors
using AbstractFFTs
using Interpolations
using DSP
using StatsBase
using Images

In [5]:
train_x, train_y = CIFAR10.traindata();
test_x, test_y   = CIFAR10.testdata();

In [6]:
function color_convert(array, color1, color2)
    chan_img = channelview(array)
    cv = colorview(color1, StackedView(array[:,:,1],array[:,:,2],array[:,:,3]))
    y_im = color2.(cv)
    channels = permutedims(channelview(float.(y_im)),(2,3,1))
    return channels
end

color_convert (generic function with 1 method)

In [8]:
test_ycc = color_convert(train_x[:,:,:,1],RGB,YCbCr)

32×32×3 Array{Float32,3}:
[:, :, 1] =
  68.5746   32.1493   36.5748   38.7416  …  164.513   141.695   145.403
  54.6376   16.0      23.6375   36.2322     148.075   126.544   133.376
  57.2476   24.6552   42.9774   68.0112     150.554   139.294   142.069
  64.7966   43.4909   64.7717   77.7211     146.838   145.396   145.951
  83.0578   66.364    84.5582   84.0506     139.924   149.825   159.223
  98.6016   92.3632  100.793    86.7765  …  131.101   153.816   174.701
 112.978    98.1422  101.651    85.1937     121.941   144.798   175.318
 116.521    96.2751   93.0116   77.4497     112.781   132.623   167.256
 121.958    97.1099   88.045    80.988      103.356   119.267   154.954
 123.862    89.9215   88.3902   93.824       97.7454  105.323   141.057
 109.103    82.1311   89.0902   98.6592  …  104.158   106.162   133.592
 105.448    79.139    85.435    85.7159     109.223   107.618   131.392
 119.348    81.7764   83.4846   87.5555     106.255   103.678   131.084
   ⋮                      

In [15]:
size(train_x)[4]

50000

In [21]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,YCbCr)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,YCbCr)
end

In [22]:
function wind_2d(nx)
    dx   = nx/2-1
    filter = zeros(Float64, nx, nx)
    A = DSP.tukey(nx, 0.3)
    itp = extrapolate(interpolate(A,BSpline(Linear())),0)
    @inbounds for x = 1:nx
        sx = x-dx-1    # define sx,sy so that no fftshift() needed
        for y = 1:nx
            sy = y-dx-1
            r  = sqrt.((sx).^2 + (sy).^2) + nx/2
            filter[x,y] = itp(r)
        end
    end
    return filter
end

function wind_2d_RGB(nx)
    dx   = nx/2-1
    filter = zeros(Float64, nx, nx)
    A = DSP.tukey(nx, 0.3)
    itp = extrapolate(interpolate(A,BSpline(Linear())),0)
    @inbounds for x = 1:nx
        sx = x-dx-1    # define sx,sy so that no fftshift() needed
        for y = 1:nx
            sy = y-dx-1
            r  = sqrt.((sx).^2 + (sy).^2) + nx/2
            filter[x,y] = itp(r)
        end
    end
    return reshape(filter,nx,nx,1)
end

function cifar_pad_RGB(im; θ=0.0)
    imbig = convert(Array{Float64,3},imresize(im,(64,64,3)))
    datad_w = fweights(wind_2d(64));
    mu_imbig = zeros(1,1,3)
    for chan = 1:3
        mu_imbig[chan] = mean(imbig[:,:,chan],datad_w)
    end
    imbig .-= mu_imbig
    imbig .*= wind_2d_RGB(64)
    impad = zeros(Float64,128,128,3)
    impad[96:-1:33,33:96,:] = imbig

    if θ != 0.0
        imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
        imrot[findall(imrot .!= imrot)] .= 0.0
        return imrot .+ mu_imbig
    end

    return impad.+ mu_imbig
end

function cifar_DHC(x)
    image = cifar_pad_RGB(x[:,:,:], θ=0.0)
    WST = DHC_compute_RGB(image, filter_hash)
    return WST
end

cifar_DHC (generic function with 1 method)

In [25]:
cifar_DHC(train_x[:,:,:,10])

LoadError: UndefVarError: DHC_compute_RGB not defined